In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:300% !important; }</style>"))

import sympy as sp

t = sp.Symbol('t')
q1 = sp.Function("q1")(t)
q2 = sp.Function("q2")(t)
q1_dot = sp.diff(q1, t)
q2_dot = sp.diff(q2, t)
q1_ddot = sp.diff(q1_dot, t)
q2_ddot = sp.diff(q2_dot, t)

# Generalized coordinates and velocities
q = [q1, q2]
q_dot = sp.Matrix([q1_dot, q2_dot])
q_ddot = sp.Matrix([q1_ddot, q2_ddot])

# Define the link lengths and masses
m, g = sp.symbols('m g')
l1, l2 = sp.symbols('l1 l2')
J1, J2 = sp.symbols('J1 J2')

In [2]:
# Actuator parameters
xa, ya = sp.symbols('xa ya') #x, y coordinate of actuator

# Rope connection length along joint 2
kx = l1 * sp.cos(q1) + l2 * sp.cos(q2)
ky = l1 * sp.sin(q1) + l2 * sp.sin(q2)
k = sp.Matrix([[kx],
               [ky]])

#R = sp.Matrix([[xa - kx],
#               [ya - ky]])
#Rx = R[0]
#Ry = R[1]
#r = R/R.norm()

l = sp.sqrt((xa - kx)**2 + (ya - ky)**2)
alpha = sp.atan2((ya - ky),(xa - kx))


A1 = sp.diff(l, q1).simplify()
A2 = sp.diff(l, q2).simplify()
A_q = sp.simplify(sp.Matrix([[A1],
                  [A2]]))

print("\n")
sp.pprint(A_q, wrap_line=False)



⎡           -l₁⋅(l₂⋅sin(q₁(t) - q₂(t)) - xa⋅sin(q₁(t)) + ya⋅cos(q₁(t)))            ⎤
⎢──────────────────────────────────────────────────────────────────────────────────⎥
⎢   _______________________________________________________________________________⎥
⎢  ╱                                     2                                       2 ⎥
⎢╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)  ⎥
⎢                                                                                  ⎥
⎢            l₂⋅(l₁⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₂(t)) - ya⋅cos(q₂(t)))            ⎥
⎢──────────────────────────────────────────────────────────────────────────────────⎥
⎢   _______________________________________________________________________________⎥
⎢  ╱                                     2                                       2 ⎥
⎣╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)  ⎦


In [3]:
h1 = l
h2 = alpha
theta = sp.Matrix([[h1],
                   [h2]])

Jh = theta.jacobian(q)
Jh = sp.simplify(Jh)
sp.pprint(Jh, wrap_line = False)

⎡           l₁⋅(-l₂⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₁(t)) - ya⋅cos(q₁(t)))                          l₂⋅(l₁⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₂(t)) - ya⋅cos(q₂(t)))            ⎤
⎢──────────────────────────────────────────────────────────────────────────────────  ──────────────────────────────────────────────────────────────────────────────────⎥
⎢   _______________________________________________________________________________     _______________________________________________________________________________⎥
⎢  ╱                                     2                                       2     ╱                                     2                                       2 ⎥
⎢╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)    ╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)  ⎥
⎢                                                                                                                                                          

# Jh_inv obtained from Mathematica code (Find_EoMs-lumped.nb)

In [4]:
A = (xa - kx)
B = (ya - ky)

Jh_inv = sp.Matrix([[(A*l2*sp.cos(q2)+B*l2*sp.sin(q2))/(sp.sqrt(A**2+B**2)*l1*l2*sp.sin(q1-q2)),
                     (A*l2*sp.sin(q2)-B*l2*sp.cos(q2))/(l1*l2*sp.sin(q1-q2))],
                   [(-A*l1*sp.cos(q1)-B*l1*sp.sin(q1))/(sp.sqrt(A**2+B**2)*l1*l2*sp.sin(q1-q2)),
                    (-A*l1*sp.sin(q1)+B*l1*sp.cos(q1))/(l1*l2*sp.sin(q1-q2))]])
Jh_inv = sp.simplify(Jh_inv)

Jh_invtrans = Jh_inv.transpose()

A_theta = Jh_invtrans * A_q
A_theta = sp.simplify(A_theta)

sp.pprint(Jh_inv, wrap_line=False)
sp.pprint(A_theta, wrap_line = False)

⎡                      -l₁⋅cos(q₁(t) - q₂(t)) - l₂ + xa⋅cos(q₂(t)) + ya⋅sin(q₂(t))                         l₁⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₂(t)) - ya⋅cos(q₂(t))⎤
⎢────────────────────────────────────────────────────────────────────────────────────────────────────────  ─────────────────────────────────────────────────────⎥
⎢      _______________________________________________________________________________                                     l₁⋅sin(q₁(t) - q₂(t))                ⎥
⎢     ╱                                     2                                       2                                                                           ⎥
⎢l₁⋅╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)  ⋅sin(q₁(t) - q₂(t))                                                       ⎥
⎢                                                                                                                                                               ⎥
⎢                       l₁ +

In [5]:
A_theta = Jh_invtrans * A_q
A_theta = sp.simplify(A_theta)
sp.pprint(A_theta, wrap_line = False)

⎡1⎤
⎢ ⎥
⎣0⎦


# Now to find the Mass, Coriolis & Centrifugal and Gravity matrix




In [6]:
# Positions of the center of mass for each link
# Defined in absolute angles from the horizontal 
x1 = l1 * sp.cos(q1)
y1 = l1 * sp.sin(q1)
x2 = l1 * sp.cos(q1) + l2 * sp.cos(q2)
y2 = l1 * sp.sin(q1) + l2 * sp.sin(q2)

# Velocities of the center of mass for each link
vx1 = sp.diff(x1, t)
vy1 = sp.diff(y1, t)
vx2 = sp.diff(x2, t)
vy2 = sp.diff(y2, t)

print(vx2)

-l1*sin(q1(t))*Derivative(q1(t), t) - l2*sin(q2(t))*Derivative(q2(t), t)


In [7]:
# Total kinetic energy
T = 0.5 * m * (vx2**2 + vy2**2)

# Total potential energy
V = m * g * y2

# Lagrangian
L = T - V

In [8]:
# Lagrangian equations
eoms = []
for i in range(2):
    L_qi = L.diff(q[i].diff(t)).diff(t) - L.diff(q[i])
    L_qi = L_qi.simplify()
    eoms.append(L_qi)


def format_lagrange(eom, name):
    eom = str(eom).replace('1.0*','')
    terms = str(eom).split('+')
    formatted = f"{name} = " 
    for i, term in enumerate(terms):
        if i == 0:
            formatted += f"{term.strip()}\n"
        else:
            formatted += f"       + {term.strip()}\n"
    return formatted

L_q1_formatted = format_lagrange(eoms[0], "L_q1")
L_q2_formatted = format_lagrange(eoms[1], "L_q2")

if __name__ == "__main__":
    # Print the formatted equations
    print(L_q1_formatted)
    print(L_q2_formatted)

print(eoms[0])

L_q1 = l1*m*(g*cos(q1(t))
       + l1*Derivative(q1(t), (t, 2))
       + l2*sin(q1(t) - q2(t))*Derivative(q2(t), t)**2
       + l2*cos(q1(t) - q2(t))*Derivative(q2(t), (t, 2)))

L_q2 = l2*m*(g*cos(q2(t)) - l1*sin(q1(t) - q2(t))*Derivative(q1(t), t)**2
       + l1*cos(q1(t) - q2(t))*Derivative(q1(t), (t, 2))
       + l2*Derivative(q2(t), (t, 2)))

1.0*l1*m*(g*cos(q1(t)) + l1*Derivative(q1(t), (t, 2)) + l2*sin(q1(t) - q2(t))*Derivative(q2(t), t)**2 + l2*cos(q1(t) - q2(t))*Derivative(q2(t), (t, 2)))


In [9]:
# Generalized velocity and acceleration
#q_dot = [q[0].diff(t), q[1].diff(t)]
#q_ddot = [q_dot[0].diff(t), q_dot[1].diff(t)]
# Mass matrix (inertia matrix)
M_q = sp.zeros(2)
for i in range(2):
    for j in range(2):
        M_q[i, j] = (T.diff(q_dot[i]).diff(q_dot[j])).simplify()
M_q = sp.nsimplify(M_q)

# Coriolis matrix
C_q = sp.zeros(2, 2)
for i in range(2):
    for j in range(2):
        for k in range(2):
            C_q[i, j] += 0.5 * (M_q[i, j].diff(q[k]) + M_q[i, k].diff(q[j]) - M_q[j, k].diff(q[i])) * q_dot[k]
C_q = sp.nsimplify(C_q)

# Gravitational force vector
G_q = sp.zeros(2, 1)
for i in range(2):
    G_q[i] = V.diff(q[i]).simplify()
G_q = sp.nsimplify(G_q)

print("M_q:")
sp.pprint(M_q)
print("\nC_q:")
sp.pprint(C_q)
print("\nG_q:")
sp.pprint(G_q)

M_q:
⎡            2                                         ⎤
⎢          l₁ ⋅m             l₁⋅l₂⋅m⋅cos(q₁(t) - q₂(t))⎥
⎢                                                      ⎥
⎢                                        2             ⎥
⎣l₁⋅l₂⋅m⋅cos(q₁(t) - q₂(t))            l₂ ⋅m           ⎦

C_q:
⎡                                                                  d        ⎤
⎢                  0                    l₁⋅l₂⋅m⋅sin(q₁(t) - q₂(t))⋅──(q₂(t))⎥
⎢                                                                  dt       ⎥
⎢                                                                           ⎥
⎢                            d                                              ⎥
⎢-l₁⋅l₂⋅m⋅sin(q₁(t) - q₂(t))⋅──(q₁(t))                   0                  ⎥
⎣                            dt                                             ⎦

G_q:
⎡g⋅l₁⋅m⋅cos(q₁(t))⎤
⎢                 ⎥
⎣g⋅l₂⋅m⋅cos(q₂(t))⎦


In [12]:

M_theta = Jh_invtrans * M_q * Jh_inv

M_theta_10 = sp.simplify(M_theta[1,0])
numerator = sp.fraction(M_theta_10)[0]
#print(numerator)#, wrap_line=False)
                        
print("WARNING: SymPy DOES NOT FIND THE PROPER SIMPLIFICATION, M_theta IS DIAGONAL FOR THIS SELECTION OF COORDINATES")
sp.pprint(sp.simplify(M_theta), wrap_line = False)

⎡                                                                                                                                                                                                                                                                                                                                                                                                                     m⋅((l₁⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₂(t)) - ya⋅cos(q₂(t)))⋅(-l₁⋅cos(q₁(t) - q₂(t)) - l₂ + xa⋅cos(q₂(t)) + ya⋅sin(q₂(t)) + (l₁ + l₂⋅cos(q₁(t) - q₂(t)) - xa⋅cos(q₁(t)) - ya⋅sin(q₁(t)))⋅cos(q₁(t) - q₂(t))) + (l₂⋅sin(q₁(t) - q₂(t)) - xa⋅sin(q₁(t)) + ya⋅cos(q₁(t)))⋅(l₁ + l₂⋅cos(q₁(t) - q₂(t)) - xa⋅cos(q₁(t)) - ya⋅sin(q₁(t)) - (l₁⋅cos(q₁(t) - q₂(t)) + l₂ - xa⋅cos(q₂(t)) - ya⋅sin(q₂(t)))⋅cos(q₁(t) - q₂(t))))⎤
⎢                                                                                                                                                                                           

In [13]:
sp.pprint(M_q)

⎡            2                                         ⎤
⎢          l₁ ⋅m             l₁⋅l₂⋅m⋅cos(q₁(t) - q₂(t))⎥
⎢                                                      ⎥
⎢                                        2             ⎥
⎣l₁⋅l₂⋅m⋅cos(q₁(t) - q₂(t))            l₂ ⋅m           ⎦


In [14]:
Jh00 = (A * l1 * sp.sin(q1) - B * l1 * sp.cos(q1)) / (sp.sqrt(A**2 + B**2))
Jh01 = (A * l2 * sp.sin(q2) - B * l2 * sp.cos(q2)) / (sp.sqrt(A**2 + B**2))
Jh10 = (- A * l1 * sp.cos(q1) - B * l1 * sp.sin(q1)) / (A**2 + B**2)
Jh11 = (- A * l2 * sp.cos(q2) - B * l2 * sp.sin(q2)) / (A**2 + B**2)


Jh_1 = sp.Matrix([[Jh00, Jh01],
                     [Jh10, Jh11]])

Jh_inv1 = sp.simplify(Jh.adjugate() / Jh.det())
Jh_invtrans1 = sp.simplify(Jh_inv1.transpose())
sp.pprint(sp.simplify(Jh_inv1))

⎡                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢───────────────────────────────────────────────────────────────────────────── ↪
⎢   ⎛    2                                                                     ↪
⎢l₁⋅⎝- l₁ ⋅sin(q₁(t) - q₂(t)) - l₁⋅l₂⋅sin(2⋅q₁(t) - 2⋅q₂(t)) + l₁⋅xa⋅sin(q₁(t) ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢───────────────────────────────────────────────────────────────────────────── ↪
⎢   ⎛    2                                                                     ↪
⎣l₂⋅⎝- l₁ ⋅sin(q₁(t) - q₂(t)

In [15]:
sp.pprint(Jh_inv - Jh_inv1, wrap_line=False)

⎡                                                                                                                                                                                                                                                              ________________________________________________________________________________________________________________________________________                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
M_theta = Jh_invtrans1 * M_q * Jh_inv1
sp.pprint(sp.simplify(M_theta), wrap_line = False)

In [ ]:
Jh_inv = sp.Matrix([[(A * l2 * sp.cos(q2) + B * l2 * sp.sin(q2))/(sp.sqrt(A**2 + B**2) * l1 * l2 * sp.sin(q1 - q2)),
                      (A * l2 * sp.sin(q2) - B * l2 * sp.cos(q2))/(l1 * l2 * sp.sin(q1 - q2))],
                   [(A * l1 * sp.cos(q1) + B * l1 * sp.sin(q1))/(sp.sqrt(A**2 + B**2) * l1 * l2 * sp.sin(q1 - q2)),
                    (A * l1 * sp.sin(q1) + B * l1 * sp.cos(q1))/(l1 * l2 * sp.sin(q1 -q2))]])

Jh_invtrans = Jh_inv.transpose()

In [ ]:
sp.pprint(sp.simplify(Jh_invtrans * M_q * Jh_inv), wrap_line = False)

In [ ]:


# Define expressions for P, Q, R, S, T, U, V
P_expr = (A * l2 * sp.cos(q2) + B * l2 * sp.sin(q2)) / (sp.sqrt(A**2 + B**2) * l1 * l2 * sp.sin(q1 - q2))
Q_expr = (A * l2 * sp.sin(q2) - B * l2 * sp.cos(q2)) / (l1 * l2 * sp.sin(q1 - q2))
R_expr = (-A * l1 * sp.cos(q1) - B * l1 * sp.sin(q1)) / (sp.sqrt(A**2 + B**2) * l1 * l2 * sp.sin(q1 - q2))
S_expr = (-A * l1 * sp.sin(q1) + B * l1 * sp.cos(q1)) / (l1 * l2 * sp.sin(q1 - q2))
T_expr = m * l1**2
U_expr = m * l1 * l2 * sp.cos(q1 - q2)
V_expr = m * l2**2

# Define matrices Jh_inv, Jh_invtrans, and M_q
Jh_inv3 = sp.Matrix([[P_expr, Q_expr], [R_expr, S_expr]])
Jh_invtrans3 = sp.Matrix([[P_expr, R_expr], [Q_expr, S_expr]])
M_q3 = sp.Matrix([[T_expr, U_expr], [U_expr, V_expr]])

sp.pprint(M_q, wrap_line=False)
sp.pprint(M_q, wrap_line=False)
sp.pprint(M_q - M_q3, wrap_line=False)

# Perform matrix multiplication and simplify
M_theta = sp.simplify(Jh_invtrans * M_q * Jh_inv)
M_theta3 = sp.simplify(Jh_invtrans * M_q3 * Jh_inv)

# Display the result
sp.pprint(M_theta, wrap_line=False)
sp.pprint(M_theta3, wrap_line=False)